<a href="https://colab.research.google.com/github/shahidnoor2217-byte/Suraksha_ai/blob/main/suraksha_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Suraksha_Ai

1.importing **libraries**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

loading **dataset**

In [ ]:
df = pd.read_csv('/content/crime_in_la.csv', on_bad_lines='skip', engine='python')
df

,DR_NO,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,LOCATION,LAT,LON
0,190326475,01/03/2020,21:30,7.0,Wilshire,784.0,510.0,VEHICLE - STOLEN,0,0.0,M,O,STREET,500.0,UNKNOWN WEAPON/OTHER WEAPON,AA,Adult Arrest,1900 S LONGWOOD AV,34.0375,-118.3506
1,200106753,08/02/2020,18:00,1.0,Central,182.0,330.0,BURGLARY FROM VEHICLE,1822 1402 0344,47.0,M,O,BUS STOP/LAYOVER (ALSO QUERY 124),500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,1000 S FLOWER ST,34.0444,-118.2628
2,200320258,04/11/2020,17:00,3.0,Southwest,356.0,480.0,BIKE - STOLEN,0344 1251,19.0,X,X,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,1400 W 37TH ST,34.0210,-118.3002
3,200907217,10/03/2020,20:37,9.0,Van Nuys,964.0,343.0,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,19.0,M,O,CLOTHING STORE,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,14000 RIVERSIDE DR,34.1576,-118.4387
4,220614831,17/08/2020,12:00,6.0,Hollywood,666.0,354.0,THEFT OF IDENTITY,1822 1501 0930 2004,28.0,M,H,SIDEWALK,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,1900 TRANSIENT,34.0944,-118.3277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139364,201516869,17/08/2020,10:50,15.0,N Hollywood,1501.0,354.0,THEFT OF IDENTITY,0377,27.0,F,W,SINGLE FAMILY DWELLING,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,8100 LLOYD AV,34.2180,-118.4207
139365,202117594,30/12/2020,20:50,21.0,Topanga,2137.0,930.0,CRIMINAL THREATS - NO WEAPON DISPLAYED,0421,29.0,F,H,SINGLE FAMILY DWELLING,511.0,VERBAL THREAT,AA,Adult Arrest,7400 VARIEL AV,34.2047,-118.5929
139366,202116836,07/12/2020,10:00,21.0,Topanga,2115.0,354.0,THEFT OF IDENTITY,1822,26.0,F,H,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,21800 LANARK ST,34.2176,-118.6045
139367,201905684,05/02/2020,06:00,19.0,Mission,1981.0,627.0,CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT,0416 0553,7.0,F,H,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",301.0,BELT FLAILING INSTRUMENT/CHAIN,AA,Adult Arrest,8700 LANGDON AV,34.2283,-118.4699


Data pre-**processing**

In [ ]:
df.columns

Index(['DR_NO', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Rpt Dist No',
       'Crm Cd', 'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex',
       'Vict Descent', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc',
       'Status', 'Status Desc', 'LOCATION', 'LAT', 'LON'],
      dtype='object')

In [ ]:
df = df[['LAT', 'LON', 'TIME OCC', 'Crm Cd Desc']]
df = df.dropna()

In [ ]:
df['TIME OCC'] = pd.to_numeric(df['TIME OCC'], errors='coerce')
df['HOUR'] = df['TIME OCC'] // 100
df.drop(columns=['TIME OCC'], inplace=True)

Fixing the **threshold**

In [ ]:
def danger_level(crime):
    crime = crime.lower()
    if 'rape' in crime or 'homicide' in crime or 'assault with deadly weapon' in crime or 'kidnap' in crime:
        return 2   # Dangerous
    elif 'robbery' in crime or 'assault' in crime or 'sexual' in crime:
        return 1   # Not Safe
    else:
        return 0   # Safe

df['danger'] = df['Crm Cd Desc'].apply(danger_level)
df.drop(columns=['Crm Cd Desc'], inplace=True)

In [ ]:
X = df[['LAT', 'LON', 'HOUR']]
y = df['danger']

Splitting the dataset into train and **test**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)

model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Testing the **accuracy**

In [ ]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.6904283561742125

Testing out **samples**

In [ ]:
# Example: Night time location
sample = [[34.0407, -118.2468, 19]]  # LAT, LON, HOUR
prediction = model.predict(sample)

labels = ['Safe', 'Not Safe', 'Dangerous']
labels[prediction[0]]

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


'Not Safe'